In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('spam').getOrCreate()

In [0]:
data = spark.read.csv('/FileStore/tables/SMSSPA_1', inferSchema=True, sep='\t')
data.show(n=5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows



In [0]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show(n=5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import length
data = data.withColumn('length', length(data['text']))
data.show(n=5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [0]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham| 71.4545266210897|
| spam|138.6706827309237|
+-----+-----------------+



In [0]:
from pyspark.ml.feature import Tokenizer,  StopWordsRemover, CountVectorizer, IDF, StringIndexer
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stopremove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='count_vec')
idf = IDF(inputCol='count_vec', outputCol='tf_idf')
class_to_num = StringIndexer(inputCol='class', outputCol='label')



In [0]:
from pyspark.ml.feature import VectorAssembler
clean_up =VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [0]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()


In [0]:
from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages = [class_to_num, tokenizer, stopremove, count_vec , idf, clean_up])
cleaner = data_prep_pipe.fit(data)


In [0]:
clean_data = cleaner.transform(data)

In [0]:
clean_data.show(n=5)

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|          token_text|          stop_token|           count_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

In [0]:
clean_data = clean_data.select('label','features')
clean_data.show(n=5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows



In [0]:
train_data , test_data = clean_data.randomSplit([0.7, 0.3])
spam_detector = nb.fit(train_data)

In [0]:
test_results = spam_detector.transform(test_data)


In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
acc

Out[18]: 0.9288872145669496